<a href="https://colab.research.google.com/github/heitorabqg/datascientist/blob/master/PSI5892_MNIST_1128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para abrir o notebook no Google Colab, altere o domínio `github.com` para `githubtocolab.com`

<div class="alert alert-block alert-danger">
Para praticar programação, é importante que você erre, leia as mensagens de erro e tente corrigí-los.
    
Dessa forma, no Google Colab, é importante que você DESATIVE OS RECURSOS DE AUTOCOMPLETAR:

- Menu Ferramentas -> Configurações
- Na janela que é aberta:
  - Seção Editor -> Desativar "Mostrar sugestões de preenchimento de código com base no contexto"
  - Seção Assistência de IA -> Desabilitar itens

Na versão em inglês:

- Menu Tools -> Settings
- Na janela que é aberta:
  - Seção Editor -> Desativar "Show context-powered code completions"
  - Seção AI Assistance -> Desabilitar itens
</div>

# PSI5892 - Aula de Exercícios

# MLP com PyTorch

Neste exercício vamos treinar uma rede MLP usando o *framework* PyTorch, para a solução de um problema de classificação usando o banco de dados [Fashion MNIST](https://arxiv.org/abs/1708.07747), que contém 70000 imagens 28x28 de peças de vestuário distribuídas em 10 classes, divididas em um conjunto de treinamento com 60000 imagens e um de teste com 10000.

Os dados estão disponíveis na biblioteca `torchvision` e os objetos `DataLoader` podem ser criados com:

``` python
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from torchvision import datasets, transforms

dir_data = "~/temp"

train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        dir_data,
        train=True,
        download=True,
        transform=transforms.Compose(            
            [transforms.ToTensor()]
        ),
    ),
    batch_size=Nb,
    shuffle=True,
)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        dir_data,
        train=False,
        transform=transforms.Compose(            
            [transforms.ToTensor()]
        ),
    ),
    batch_size=Nb_test,
    shuffle=True,
)
```

Vale notar alguns detalhes sobre o código anterior:

 - o `DataLoader` de treinamento é criado com `train=True` e o de teste, com `train=False`, o que garante que não haja dados em comum entre os dois conjuntos;
 - `Nb` e `Nb_test` representam os tamanhos dos mini *batches* de treino e teste;
 - É feita a configuração de uma transformação de dados ao carregá-los. Para isso, é criado um objeto do tipo `transforms.Compose`, que permite encadear uma série de transformações a serem aplicadas às imagens, durante o carregamento. Nesse caso, a transformação tem uma única etapa que consistem em converter os valores obtidos para um tensor do PyTorch.

Para ver algumas imagens do banco de dados usando o DataLoader criado, pode ser utilizado o seguinte código:

``` python
plt.figure(figsize=(16, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    image, _ = train_loader.dataset.__getitem__(i)
    plt.imshow(image.squeeze().numpy())
    plt.axis('off');
```


# Exercício 1

Implemente uma rede MLP para classificação de imagens do conjunto Fashion MNIST usando o PyTorch. Lembre-se que trata-se de um problema de classificação multiclasse e utilize a arquitetura mais adequada.

No caso de usar a entropia cruzada, vale notar que a função custo `CrossEntropyLoss` espera comparar um vetor de $C$ posições com um número de $0$ a $C-1$, conforme descrito na [documentação](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html). Além disso, é esperado que os elementos do vetor representem a *evidência*, ou seja os valores chamados de *logits*, que não são normalizados e podem valer de $-\infty$ a $\infty$. Por isso, na saída da rede, não é usada a função *Softmax*.

Por fim, avalie o modelo treinado em termos de acurácia (número de acertos dividido pelo número de testes) e busque variar os hiperparâmetros da rede a fim de obter uma acurácia próxima a 85%.

## Resolução

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# --- Hiperparâmetros ---
Nb = 64
Nb_test = 256
lr = 0.001
epochs = 10

# --- Carregar dados ---
dir_data = "./data"

transform = transforms.Compose([transforms.ToTensor()])

train_loader = DataLoader(
    datasets.FashionMNIST(
        dir_data,
        train=True,
        download=True,
        transform=transform
    ),
    batch_size=Nb,
    shuffle=True
)

test_loader = DataLoader(
    datasets.FashionMNIST(
        dir_data,
        train=False,
        transform=transform
    ),
    batch_size=Nb_test,
    shuffle=False
)

# --- Definição da MLP ---
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.net = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10)    # logits (sem softmax!)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.net(x)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# --- Função de treino ---
def train(model, loader):
    model.train()
    total_loss = 0

    for X, y in loader:
        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

# --- Função de avaliação ---
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in loader:
            logits = model(X)
            preds = torch.argmax(logits, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total

# --- Loop de treinamento ---
for epoch in range(epochs):
    loss = train(model, train_loader)
    acc = evaluate(model, test_loader)
    print(f"Época {epoch+1}/{epochs} - Loss: {loss:.4f} - Acurácia: {acc*100:.2f}%")

print("\nTreinamento concluído!")


acc_final = evaluate(model, test_loader)
print(f"\nAcurácia final no conjunto de teste: {acc_final*100:.2f}%")


100%|██████████| 26.4M/26.4M [00:01<00:00, 13.3MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 211kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.92MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.2MB/s]


Época 1/10 - Loss: 0.4968 - Acurácia: 84.80%
Época 2/10 - Loss: 0.3592 - Acurácia: 86.18%
Época 3/10 - Loss: 0.3242 - Acurácia: 87.52%
Época 4/10 - Loss: 0.3005 - Acurácia: 87.63%
Época 5/10 - Loss: 0.2779 - Acurácia: 87.78%
Época 6/10 - Loss: 0.2676 - Acurácia: 88.08%
Época 7/10 - Loss: 0.2506 - Acurácia: 88.33%
Época 8/10 - Loss: 0.2392 - Acurácia: 88.62%
Época 9/10 - Loss: 0.2273 - Acurácia: 87.21%
Época 10/10 - Loss: 0.2196 - Acurácia: 87.77%

Treinamento concluído!

Acurácia final no conjunto de teste: 87.77%
